# Reduced Data Structures
This notebook chooses a small consistent subset of the treated dataset to be run in small machines. The generated dataset is limited to a number of **drug-drug interactions** specified by the variable $N$. From this dataset, it generates the corresponding data structures like dictionaries and adjacency matrices to be fed directly to DECAGON.<br>
This code is in part the adaptation in `pandas` of the script `drug_dataset.sh`, merged with `data_structures.ipynb`.

Author: Juan Sebastian Diaz Boada, May 2020

### Python 3

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import scipy.sparse as sp
import shelve
import pandas as pd

In [3]:
# Number of side effects
N = 3

### Import databases

In [4]:
PPI = pd.read_csv('clean_data/new-decagon-ppi.csv',sep=',')
PF = pd.read_csv('clean_data/new-decagon-genes.csv',sep=',')
DTI = pd.read_csv('clean_data/new-decagon-targets.csv',sep=',')
DDI = pd.read_csv('clean_data/new-decagon-combo.csv',sep=',')
DSE = pd.read_csv('clean_data/new-decagon-mono.csv',sep=',')
SE = pd.read_csv('original_data/bio-decagon-effectcategories.csv',sep=',')

In [5]:
# Number of interactions
orig_ppi = len(PPI.index)
orig_pf = len(PF.index)
orig_dti = len(DTI.index)
orig_ddi = len(DDI.index)
orig_dse = len(DSE.index)
# Number of nodes
orig_ddi_drugs = len(pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel()))
orig_ppi_genes = len(pd.unique(PPI[['Gene 1','Gene 2']].values.ravel()))
orig_dti_drugs = len(pd.unique(DTI['STITCH']))
orig_dti_genes = len(pd.unique(DTI['Gene']))
orig_se_mono = len(pd.unique(DSE['Side Effect Name']))

### Choose Side effects

In [6]:
se = SE.sample(n=N, axis=0)['Side Effect'].values

### Select DDIs

In [7]:
DDI = DDI[DDI['Polypharmacy Side Effect'].isin(se)].reset_index(drop=True)
DDI_drugs = pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel()) # Unique drugs 
drug2idx = {drug: i for i, drug in enumerate(DDI_drugs)}
se_names = pd.unique(DDI['Side Effect Name']) # Unique joint side effects
se_combo_name2idx = {se: i for i, se in enumerate(se_names)}
n_drugs = len(DDI_drugs)

### Select Drug side effects

In [8]:
DSE = DSE[DSE['STITCH'].isin(DDI_drugs)].reset_index(drop=True)
se_mono_names = pd.unique(DSE['Side Effect Name'].values) # Unique individual side effects
se_mono_name2idx = {name: i for i, name in enumerate(se_mono_names)}
n_semono = len(se_mono_names)

### Select DTIs

In [9]:
DTI = DTI[DTI['STITCH'].isin(DDI_drugs)].reset_index(drop=True)
DTI_genes = pd.unique(DTI['Gene']) # Unique genes in DTI
DTI_drugs = pd.unique(DTI['STITCH']) # Unique drugs in DTI
dti_drugs = len(DTI_drugs)
dti_genes = len(DTI_genes)

### Select PPIs

In [10]:
PPI = PPI[np.logical_or(PPI['Gene 1'].isin(DTI_genes),
                       PPI['Gene 2'].isin(DTI_genes))].reset_index(drop=True)
PPI_genes = pd.unique(PPI[['Gene 1','Gene 2']].values.ravel()) # Unique genes is PPI
gene2idx = {gene: i for i, gene in enumerate(PPI_genes)}
n_genes = len(PPI_genes)

### Select PFs

In [11]:
PF = PF[PF['GeneID'].isin(PPI_genes)].reset_index(drop=True)

## Adjacency matrices and degrees

In [14]:
# DDI adjacency matrices and degrees
ddi_adj_list = []
for i in se_combo_name2idx.keys():
    m = np.zeros([n_drugs,n_drugs],dtype=int)
    seDDI = DDI[DDI['Side Effect Name'].str.match(i)].reset_index()
    for j in seDDI.index:
        row = drug2idx[seDDI.loc[j,'STITCH 1']]
        col = drug2idx[seDDI.loc[j,'STITCH 2']]
        m[row,col] = m[col,row] = 1
    ddi_adj_list.append(sp.csr_matrix(m))
ddi_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in ddi_adj_list]

In [15]:
# DTI adjacency matrix
dti_adj = np.zeros([n_genes,n_drugs],dtype=int)
for i in DTI.index:
    row = gene2idx[DTI.loc[i,'Gene']]
    col = drug2idx[DTI.loc[i,'STITCH']]
    dti_adj[row,col] = 1
dti_adj = sp.csr_matrix(dti_adj)

In [13]:
# PPI adjacency matrix and degrees
ppi_adj = np.zeros([n_genes,n_genes],dtype=int)
for i in PPI.index:
    row = gene2idx[PPI.loc[i,'Gene 1']]
    col = gene2idx[PPI.loc[i,'Gene 2']]
    ppi_adj[row,col]=ppi_adj[col,row]=1
ppi_degrees = np.sum(ppi_adj,axis=0)
ppi_adj = sp.csr_matrix(ppi_adj)

In [16]:
# Drug Feature matrix
drug_feat = np.zeros([n_drugs,n_semono],dtype=int)
for i in DSE.index:
    row = drug2idx[DSE.loc[i,'STITCH']]
    col = se_mono_name2idx[DSE.loc[i,'Side Effect Name']]
    drug_feat[row,col] = 1
drug_feat = sp.csr_matrix(drug_feat)

In [12]:
# Protein feature matrices
prot_feat = sp.coo_matrix(
    PF[['GeneID', 'Length', 'Mass', 'n_helices', 'n_strands', 'n_turns']].to_numpy())
norm_prot_feat = sp.coo_matrix(
    PF[['Normalized Helices(Mean)', 'Normalized Helices(Median)',
       'Normalized Strands(Mean)', 'Normalized Strands(Median)',
       'Normalized Turns(Mean)', 'Normalized Turns(Median)']].to_numpy())

### Print

In [17]:
print('Original number of PPI interactions:', orig_ppi)
print('New number of PPI interactions:', len(PPI.index))
print('\n')
print('Original number of DDI interactions:', orig_ddi)
print('New number of DDI interactions:', len(DDI.index))
print('\n')
print('Original number of DTI interactions:', orig_dti)
print('New number of DTI interactions:', len(DTI.index))
print('Original number of DTI genes:', orig_dti_genes)
print('New number of DTI genes:',len(pd.unique(DTI['Gene'].values)))
print('Original number of DTI drugs:', orig_dti_drugs)
print('New number of DTI drugs:',len(pd.unique(DTI['STITCH'].values)))
print('\n')
print('Original number of DSE interactions:', orig_dse)
print('New number of DSE interactions:', len(DSE.index))
print('\n')
print('Original number of single side effects:', orig_se_mono)
print('New number of single side effects:', n_semono)
print('\n')
print('Original number of proteins with features:', orig_pf)
print('New number of proteins with features:', len(PF.index))
print('\n')
print('Original number of genes:',orig_ppi_genes)
print('New number of genes:', n_genes)
print('\n')
print('Original number of drugs:',orig_ddi_drugs)
print('New number of drugs:', n_drugs)

Original number of PPI interactions: 693353
New number of PPI interactions: 310064


Original number of DDI interactions: 4615522
New number of DDI interactions: 15565


Original number of DTI interactions: 18291
New number of DTI interactions: 16989
Original number of DTI genes: 3587
New number of DTI genes: 3564
Original number of DTI drugs: 283
New number of DTI drugs: 240


Original number of DSE interactions: 174977
New number of DSE interactions: 161234


Original number of single side effects: 9702
New number of single side effects: 9581


Original number of proteins with features: 17929
New number of proteins with features: 16267


Original number of genes: 17929
New number of genes: 16267


Original number of drugs: 639
New number of drugs: 539


## Save

In [18]:
now = datetime.datetime.now() # current date and time
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
filename = './data_structures/decagon_se'+str(N)+'_'+year+'-'+month+'-'+day
data = shelve.open(filename,'n',protocol=2)

In [19]:
# Dictionaries
data['gene2idx'] = gene2idx
data['drug2idx'] = drug2idx
data['se_mono_name2idx'] = se_mono_name2idx
data['se_combo_name2idx'] = se_combo_name2idx
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat
# PF
data['prot_feat'] = prot_feat
data['norm_prot_feat'] = norm_prot_feat

In [20]:
data.close()